## POS tagging using modified Viterbi

#### Flowchart of the solution -

<ul>
    <li>EDA to understand training corpus.</li>
    <li>Plain vanilla model building.</li>
    <li>Test plain vanilla model on test set and understand the problem.</li>
    <li>Refining viterbi model using other pos tagging technique</li>
</ul>

### Data Preparation

In [1]:
#Importing libraries
#Importing libraries
import nltk, re, pprint
import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import pprint, time
import random
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize

In [2]:
# reading the Treebank tagged sentences
nltk_data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))

In [3]:
# Splitting into train and test
# we will consider train : test ratio of 95:5 as suggested in assignment
random.seed(1234)
train_set, test_set = train_test_split(nltk_data,test_size=0.05)

print(len(train_set))
print(len(test_set))


3718
196


In [4]:
# Getting list of tagged words
train_tagged_words = [tup for sent in train_set for tup in sent]
len(train_tagged_words)

95809

In [5]:
#no of tagged words available in dataset - 95790

In [6]:
#Let's see how many unique tags we have in our dataset
tags = [tup[1]  for sen in nltk_data for tup in sen]
print(len(set(tags)))
print(set(tags))


12
{'NOUN', 'ADP', '.', 'DET', 'ADJ', 'PRON', 'NUM', 'PRT', 'VERB', 'X', 'CONJ', 'ADV'}


In [46]:
#we can see that universal dataset has only 12 tags
#Let's see how many unique word dataset has
voc = [tup[0]  for sen in nltk_data for tup in sen]
print("Considering duplicate -",len(voc))
#total no. words present in dataset (including duplicate) 
print("Unique words -",len(set(voc)))
#total no. of unique words

Considering duplicate - 100676
Unique words - 12408


In [47]:
tags = set(tags)
voc = set(voc)

In [48]:
#print all the available tags
print(tags)

{'NOUN', 'ADP', '.', 'DET', 'ADJ', 'PRON', 'NUM', 'PRT', 'VERB', 'X', 'CONJ', 'ADV'}


In [55]:
#this method we'll use to understand tags related with words
def plot_cnt_words(word):
    l_words = []
    for tag in tags:
        c = 0
        for w,t in train_tagged_words:
            if (w==word)&(t==tag):
                    c += 1
        if c > 0:
            l_words.append((tag,c))
    print(l_words)
        

In [56]:
#demo
plot_cnt_words("and")

[('NOUN', 2), ('ADP', 1), ('ADJ', 3), ('CONJ', 1423)]


### Build the vanilla Viterbi based POS tagger
Let's build HMM viterbi model. (This model will act as a base model)

In [57]:
#first step is to find emission and transition probablities
t = len(tags)
v = len(voc)
w_given_t = np.zeros((t, v))

In [58]:
w_given_t.shape

(12, 12408)

In [59]:
# compute word given tag: Emission Probability
def word_given_tag(word, tag, train_bag = train_tagged_words):
    tag_list = [pair for pair in train_bag if pair[1]==tag]
    count_tag = len(tag_list)
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
    count_w_given_tag = len(w_given_tag_list)
    
    return (count_w_given_tag, count_tag)

In [62]:
# let's check w
print(word_given_tag('and', 'ADJ'))
print(word_given_tag('does', 'VERB'))
print(word_given_tag('flight', 'NOUN'), "\n")
#flight word is not present in the dictionary

(3, 6084)
(52, 12904)
(0, 27480) 



In [63]:
# compute tag given tag: tag2(t2) given tag1 (t1), i.e. Transition Probability

def t2_given_t1(t2, t1, train_bag = train_tagged_words):
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t==t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)

In [65]:
# examples
print(t2_given_t1(t2='NOUN', t1='CONJ'))

(744, 2142)


In [66]:
# creating t x t transition matrix of tags
# each column is t2, each row is t1
# thus M(i, j) represents P(tj given ti)

tags_matrix = np.zeros((len(tags), len(tags)), dtype='float32')
for i, t1 in enumerate(list(tags)):
    for j, t2 in enumerate(list(tags)): 
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]

In [67]:
tags_matrix

array([[2.64519662e-01, 1.76491991e-01, 2.40393013e-01, 1.32096075e-02,
        1.21906837e-02, 4.54876292e-03, 9.60698724e-03, 4.37772907e-02,
        1.47161573e-01, 2.92940326e-02, 4.19941768e-02, 1.68122277e-02],
       [3.21725249e-01, 1.75718851e-02, 3.94036211e-02, 3.22151214e-01,
        1.07774228e-01, 6.95420653e-02, 6.27263039e-02, 1.38445152e-03,
        8.20021331e-03, 3.50372754e-02, 8.51970166e-04, 1.36315227e-02],
       [2.22669885e-01, 9.13242027e-02, 9.39206704e-02, 1.72799706e-01,
        4.38714288e-02, 6.50908798e-02, 8.13859776e-02, 2.14880472e-03,
        8.86381939e-02, 2.70391256e-02, 5.91816641e-02, 5.18399142e-02],
       [6.38747871e-01, 9.30626038e-03, 1.83707997e-02, 5.55958413e-03,
        2.03770846e-01, 3.50495521e-03, 2.17548944e-02, 2.41721049e-04,
        3.96422520e-02, 4.58061397e-02, 4.83442098e-04, 1.28112156e-02],
       [7.01347768e-01, 7.79092684e-02, 6.34450987e-02, 4.93096653e-03,
        6.60749525e-02, 6.57462224e-04, 2.05456931e-02, 1.10

In [68]:
# convert the matrix to a df for better readability
tags_df = pd.DataFrame(tags_matrix, columns = list(tags), index=list(tags))

In [69]:
#this dataframe is usefull to calculate tag probablities
tags_df

,NOUN,ADP,.,DET,ADJ,PRON,NUM,PRT,VERB,X,CONJ,ADV
NOUN,0.264520,0.176492,0.240393,0.013210,0.012191,0.004549,0.009607,0.043777,0.147162,0.029294,0.041994,0.016812
ADP,0.321725,0.017572,0.039404,0.322151,0.107774,0.069542,0.062726,0.001384,0.008200,0.035037,0.000852,0.013632
.,0.222670,0.091324,0.093921,0.172800,0.043871,0.065091,0.081386,0.002149,0.088638,0.027039,0.059182,0.051840
DET,0.638748,0.009306,0.018371,0.005560,0.203771,0.003505,0.021755,0.000242,0.039642,0.045806,0.000483,0.012811
ADJ,0.701348,0.077909,0.063445,0.004931,0.066075,0.000657,0.020546,0.011012,0.012492,0.020546,0.016108,0.004931
PRON,0.209133,0.023024,0.041059,0.009210,0.072525,0.008058,0.007675,0.013047,0.486953,0.089409,0.005372,0.034536
NUM,0.353723,0.033392,0.117317,0.003251,0.032210,0.001478,0.186170,0.027482,0.017730,0.211879,0.012707,0.002660
PRT,0.244618,0.020222,0.043053,0.101435,0.085453,0.018591,0.057404,0.001957,0.401500,0.014025,0.001957,0.009785
VERB,0.110586,0.091754,0.034795,0.133757,0.065716,0.035415,0.022629,0.031696,0.168630,0.217607,0.005347,0.082068
X,0.062589,0.145512,0.163463,0.054964,0.017156,0.056394,0.002859,0.184432,0.202859,0.074186,0.010008,0.025576


## Viterbi Algorithm - Simple

In [70]:
len(train_tagged_words)

95809

In [23]:
# Viterbi Heuristic
def Viterbi(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

## 4. Evaluating on Test Set

In [24]:
# Running on entire test dataset would take more than 3-4hrs. 
# Let's test our Viterbi algorithm on a few sample sentences of test dataset

random.seed(1234)

# choose random 5 sents
rndom = [random.randint(1,len(test_set)) for x in range(5)]

# list of sents
test_run = [test_set[i] for i in rndom]

# list of tagged words
test_run_base = [tup for sent in test_run for tup in sent]

# list of untagged words
test_tagged_words = [tup[0] for sent in test_run for tup in sent]
test_run

[[('Profit', 'NOUN'),
  (',', '.'),
  ('at', 'ADP'),
  ('least', 'ADJ'),
  ('in', 'ADP'),
  ('the', 'DET'),
  ('short', 'ADJ'),
  ('term', 'NOUN'),
  (',', '.'),
  ('is', 'VERB'),
  ('usually', 'ADV'),
  ('a', 'DET'),
  ('secondary', 'ADJ'),
  ('goal', 'NOUN'),
  ('.', '.')],
 [('School', 'NOUN'),
  ('officials', 'NOUN'),
  ('and', 'CONJ'),
  ('prosecutors', 'NOUN'),
  ('say', 'VERB'),
  ('0', 'X'),
  ('Mrs.', 'NOUN'),
  ('Yeargin', 'NOUN'),
  ('is', 'VERB'),
  ('lying', 'VERB'),
  ('.', '.')],
 [('They', 'PRON'),
  ('call', 'VERB'),
  ('it', 'PRON'),
  ('``', '.'),
  ('photographic', 'ADJ'),
  ("''", '.'),
  ('.', '.')],
 [('Her', 'PRON'),
  ('alternative', 'NOUN'),
  ('was', 'VERB'),
  ('90', 'NUM'),
  ('days', 'NOUN'),
  ('in', 'ADP'),
  ('jail', 'NOUN'),
  ('.', '.')],
 [('Under', 'ADP'),
  ('a', 'DET'),
  ('1934', 'NUM'),
  ('law', 'NOUN'),
  (',', '.'),
  ('the', 'DET'),
  ('Johnson', 'NOUN'),
  ('Debt', 'NOUN'),
  ('Default', 'NOUN'),
  ('Act', 'NOUN'),
  (',', '.'),
  ('as', 'A

In [25]:
# tagging the test sentences
start = time.time()
tagged_seq = Viterbi(test_tagged_words)
end = time.time()
difference = end-start

In [26]:
print("Time taken in seconds: ", difference)
print(tagged_seq)
#print(test_run_base)

Time taken in seconds:  18.63800048828125
[('Profit', 'NOUN'), (',', '.'), ('at', 'ADP'), ('least', 'ADJ'), ('in', 'ADP'), ('the', 'DET'), ('short', 'ADJ'), ('term', 'NOUN'), (',', '.'), ('is', 'VERB'), ('usually', 'ADV'), ('a', 'DET'), ('secondary', 'ADJ'), ('goal', 'NOUN'), ('.', '.'), ('School', 'NOUN'), ('officials', 'NOUN'), ('and', 'CONJ'), ('prosecutors', 'NOUN'), ('say', 'VERB'), ('0', 'X'), ('Mrs.', 'NOUN'), ('Yeargin', 'NOUN'), ('is', 'VERB'), ('lying', 'VERB'), ('.', '.'), ('They', 'PRON'), ('call', 'VERB'), ('it', 'PRON'), ('``', '.'), ('photographic', 'NOUN'), ("''", '.'), ('.', '.'), ('Her', 'PRON'), ('alternative', 'NOUN'), ('was', 'VERB'), ('90', 'NUM'), ('days', 'NOUN'), ('in', 'ADP'), ('jail', 'NOUN'), ('.', '.'), ('Under', 'ADP'), ('a', 'DET'), ('1934', 'NOUN'), ('law', 'NOUN'), (',', '.'), ('the', 'DET'), ('Johnson', 'NOUN'), ('Debt', 'NOUN'), ('Default', 'NOUN'), ('Act', 'NOUN'), (',', '.'), ('as', 'ADP'), ('*', 'X'), ('amended', 'NOUN'), ('*-1', 'X'), (',', '.'), 

In [27]:
# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 

In [28]:
accuracy = len(check)/len(tagged_seq)

In [29]:
accuracy
#92#94.62

0.9431818181818182

In [30]:
incorrect_tagged_cases = [[test_run_base[i-1],j] for i, j in enumerate(zip(tagged_seq, test_run_base)) if j[0]!=j[1]]

In [31]:
incorrect_tagged_cases

[[('``', '.'), (('photographic', 'NOUN'), ('photographic', 'ADJ'))],
 [('a', 'DET'), (('1934', 'NOUN'), ('1934', 'NUM'))],
 [(',', '.'), (('as', 'ADP'), ('as', 'ADV'))],
 [('*', 'X'), (('amended', 'NOUN'), ('amended', 'VERB'))],
 [('*EXP*-2', 'X'), (("'s", 'PRT'), ("'s", 'VERB'))]]

In [32]:
#Now let's test this model on test sentences which contains words which are not present in the training dataset.

In [33]:
## Testing

def test_vertibi_simple(sentence):
    words = word_tokenize(sentence)
    tagged_seq = Viterbi(words)
    return tagged_seq

In [34]:
test_vertibi_simple("Android has been the best-selling OS worldwide on smartphones since 2011 and on tablets since 2013.")

[('Android', 'NOUN'),
 ('has', 'VERB'),
 ('been', 'VERB'),
 ('the', 'DET'),
 ('best-selling', 'ADJ'),
 ('OS', 'NOUN'),
 ('worldwide', 'NOUN'),
 ('on', 'ADP'),
 ('smartphones', 'NOUN'),
 ('since', 'ADP'),
 ('2011', 'NOUN'),
 ('and', 'CONJ'),
 ('on', 'ADP'),
 ('tablets', 'NOUN'),
 ('since', 'ADP'),
 ('2013', 'NOUN'),
 ('.', '.')]

In [35]:
#Android is noun but incorrectly tagged as PRT, as emission prob for Android will be 0 and hence it will assign tag which has 
#o value. Let's see two more exaples.

In [36]:
test_vertibi_simple("Google and Twitter made a deal in 2015 that gave Google access to Twitter's firehose.")

[('Google', 'NOUN'),
 ('and', 'CONJ'),
 ('Twitter', 'NOUN'),
 ('made', 'VERB'),
 ('a', 'DET'),
 ('deal', 'NOUN'),
 ('in', 'ADP'),
 ('2015', 'NOUN'),
 ('that', 'ADP'),
 ('gave', 'VERB'),
 ('Google', 'NOUN'),
 ('access', 'NOUN'),
 ('to', 'PRT'),
 ('Twitter', 'NOUN'),
 ("'s", 'PRT'),
 ('firehose', 'NOUN'),
 ('.', '.')]

In [37]:
#Same problem for google and twitter

In [38]:
test_vertibi_simple("NASA invited social media users to experience the launch of ICESAT-2 Satellite.")

[('NASA', 'NOUN'),
 ('invited', 'NOUN'),
 ('social', 'ADJ'),
 ('media', 'NOUN'),
 ('users', 'NOUN'),
 ('to', 'PRT'),
 ('experience', 'NOUN'),
 ('the', 'DET'),
 ('launch', 'NOUN'),
 ('of', 'ADP'),
 ('ICESAT-2', 'NOUN'),
 ('Satellite', 'NOUN'),
 ('.', '.')]

In [39]:
#same

In [40]:
#So we need to use other techni#sameques to rectify this issue

### Solve the problem of unknown words

In [41]:
# Viterbi Heuristic
def Viterbi(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        if max(p) == 0 :
            state_max = T[3]
        if word.isnumeric() & (word != '0'):
            state_max = T[10]
        state.append(state_max)
    return list(zip(words, state))

In [73]:
Viterbi(['Twitter'])

[('Twitter', 'DET')]

#### Evaluating tagging accuracy

### Compare the tagging accuracies of the modifications with the vanilla Viterbi algorithm

### List down cases which were incorrectly tagged by original POS tagger and got corrected by your modifications